In [10]:
import importlib
# import engine
# import nn
# importlib.reload(engine)
# importlib.reload(nn)
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


In [11]:
X, y = load_digits(return_X_y=True)

In [12]:
t = MinMaxScaler()
t.fit(X)
X_train = t.transform(X)

X_train

array([[0.    , 0.    , 0.3125, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.625 , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 1.    , 0.5625, 0.    ],
       ...,
       [0.    , 0.    , 0.0625, ..., 0.375 , 0.    , 0.    ],
       [0.    , 0.    , 0.125 , ..., 0.75  , 0.    , 0.    ],
       [0.    , 0.    , 0.625 , ..., 0.75  , 0.0625, 0.    ]])

In [13]:


# def loss(batch_size):
#     epsilon = 1e-8
#     ri = np.random.permutation(X.shape[0])[:batch_size]
#     Xb = X[ri]
#     ri = np.random.permutation(y.shape[0])[:batch_size]
#     yb = y[ri]
    
#     inputs = [list(map(engine.Value, xrow)) for xrow in Xb]
    
#     print("is me", inputs)
    
#     scores = [n(input) for input in inputs]
    
    
#     scores = [i.sigmoid() for i in scores]
    
#     combination = zip(scores, yb)
# # 
#     combination = [i for i in combination if not np.isnan(i[0].data)]
    

#     Binary_Cross_Entropy = [(truth * ((y + epsilon).log()) + (1 - truth) * ((1 - y + epsilon).log())) for y, truth in combination]
    
#     loss = sum(Binary_Cross_Entropy) * -(1/len(scores))
    
#     alpha = 1e-4
#     reg_loss = alpha * sum((p*p for p in n.parameters()))
    
    
#     total_loss = loss + reg_loss
    
#     accuracy = [(yi > 0) == (scorei.data > 0) for scorei, yi in combination]
    
#     return total_loss, sum(accuracy) / len(accuracy)

# print(loss(15))
        

In [14]:
# # optimization
# for k in range(100):
    
#     # forward
#     total_loss, acc = loss(100)
    
#     # backward
#     n.zero_grad()
#     total_loss.backward()
    
#     # update (sgd)
#     learning_rate = 1.0 - 0.9*k/100
#     for p in n.parameters():
#         p.data -= learning_rate * p.grad
    
#     if k % 1 == 0:
#         print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


In [15]:
import math
import numpy as np


class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op  # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)
                          ), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    # fix dead neuron problem
    def leaky_relu(self):
        out = Value(self.data * 0.01 if self.data <=
                    0 else self.data, (self,), 'ReLU')

        def _backward():
            local_grad = 1.0 if self.data > 0 else 0.01
            self.grad += local_grad * out.grad 
        out._backward = _backward
        return out

    def log(self):

        out = Value(math.log(self.data), (self, ), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def sigmoid(self):
        x = self.data
        t = 1 / (1 + (np.exp(-x)))

        out = Value(t, (self, ), 'sigmoid')

        def _backward():
            self.grad += (out.data * (1 - out.data)) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __ge__(self, other):
        return self.data >= other.data

    def __le__(self, other):
        return self.data <= other.data

    def __gt__(self, other):
        return self.data > other.data

    def __lt__(self, other):
        return self.data < other.data

    def __neg__(self):  # -self
        return self * -1

    def __radd__(self, other):  # other + self
        return self + other

    def __sub__(self, other):  # self - other
        return self + (-other)

    def __rsub__(self, other):  # other - self
        return other + (-self)

    def __rmul__(self, other):  # other * self
        return self * other

    def __truediv__(self, other):  # self / other
        return self * other**-1

    def __rtruediv__(self, other):  # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"


In [16]:
import random
from graphviz import Digraph
import math
import importlib


class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def zero_weights(self):
        for w in self.weights():
            for j in w:
                print(j)
                j.grad = 0

    def parameters(self):
        return []

    def weights(self):
        return []

    def layers(self):
        return []

    def summary(self):
        return f"{len(self.layers())} layers, {len(self.parameters)} parameters"


class Neuron(Module):

    # I want to introduce weight sharing, which means I need to be able to
    # initialise a neuron with pre defined weights, but leave the bias?

    def __init__(self, nin, nonlin=True, **kwargs):
        tied_weights = kwargs.get('tied_weights', None)
        self.w = tied_weights if tied_weights is not None else [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        self.nonlin = nonlin
        self.activate = kwargs.get('activate', None)

    def __call__(self, x):
        # print("come on then", self.w, "come on son                   ",x)
        if isinstance(x, Value):
            # This is for a single input, likely at the start of a layer
            act = self.w[0] * x + self.b
        else:
            act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        if self.activate and self.nonlin == False:
            return self.activate(act)
        else:
            return act.leaky_relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b] if isinstance(self.w[0], Value) else [p for w_list in self.w for p in w_list] + [self.b]

    def weights(self):
        # return self.w without the brackets
        return [self.w]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else '{self.activate}'}Neuron({len(self.w)})"


class Layer(Module):
    def __init__(self, nin, nout, tied_to_layer=None, **kwargs):
        if tied_to_layer is None:
            # Standard Layer initialization
            self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]
        else:
            # Tied Layer initialization
            # The weights for this layer are the transpose of the tied_to_layer's weights
            # This requires careful construction.
            # Number of inputs for this layer = number of outputs of the tied layer
            # Number of outputs for this layer = number of inputs of the tied layer
            self.tied_to_layer = tied_to_layer
            self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]
            
    def __call__(self, x):
        if hasattr(self, 'tied_to_layer'):
            # The weights are conceptually transposed.
            # So, the output of a neuron is sum(w_ji * x_i), which means summing over the neurons of the previous layer.
            # This is hard to do cleanly with the current structure.
            # The simpler approach is to loop manually.
            out = []
            for j in range(len(self.neurons)):
                # The j-th neuron of this layer uses the j-th weight of every neuron in the tied layer.
                # For each output neuron (j), sum the weighted inputs.
                # The weight connecting input `i` to output `j` is the same as the weight connecting
                # input `j` of the encoder layer to output `i`.
                act = sum(self.tied_to_layer.neurons[i].w[j] * x[i] for i in range(len(x))) + self.neurons[j].b if isinstance(x, list) else self.tied_to_layer.neurons[0].w[j] * x + self.neurons[j].b
                
                # Apply activation
                if self.neurons[j].activate and self.neurons[j].nonlin is False:
                    act = self.neurons[j].activate(act)
                else:
                    act = act.relu() if self.neurons[j].nonlin else act
                out.append(act)
            return out[0] if len(out) == 1 else out
        else:
            # Standard layer behavior
            out = [n(x) for n in self.neurons]
            return out[0] if len(out) == 1 else out

    def parameters(self):
        # In a tied layer, the weights are shared, but the biases are not.
        if hasattr(self, 'tied_to_layer'):
            return [n.b for n in self.neurons]
        else:
            return [p for n in self.neurons for p in n.parameters()]
                

    def weights(self):
        if hasattr(self, 'tied_to_layer'):
            # The weights belong to the encoder, so we don't need to report them here
            # as they are included in the encoder's parameters list.
            return []
        else:
            return [p for n in self.neurons for p in n.weights()]
        
    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"


class MLP(Module):
    def __init__(self, nin, nouts, tied_weights_from=None, **kwargs):
        sz = [nin] + nouts
        self.layers = []
        if tied_weights_from is None:
            # Standard MLP initialization
            self.layers = [Layer(sz[i], sz[i+1], nonlin=i != len(nouts)-1, **kwargs) for i in range(len(nouts))]
        else:
            # Tied-weight MLP initialization
            tied_layers = list(reversed(tied_weights_from))
            for i in range(len(nouts)):
                # Pass the encoder's layer directly to the decoder's layer.
                # The decoder layer will use the encoder's weights.
                self.layers.append(Layer(sz[i], sz[i+1], tied_to_layer=tied_layers[i], nonlin=i != len(nouts)-1, **kwargs))

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def weights(self):
        # [[p for p in layer.weights()] for layer in self.layers] 
        return [p for layer in self.layers for p in layer.weights()]
        

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

class AutoEncoder(Module):
    def __init__(self, in_embeds, n_hidden_layers, compressed, act_func=None, tied=False):
        n_hidden_layers = [math.ceil(in_embeds / i)
                           for i in range(2, n_hidden_layers + 2)]
        
        
        self.act_func = act_func
        
        # Create encoder
        self.encoder = MLP(in_embeds, n_hidden_layers + [compressed])
        
        
        
        # Create decoder, passing encoder layers for tied weights
        if tied:
            self.decoder = MLP(compressed, list(reversed(n_hidden_layers)) + [in_embeds], tied_weights_from=self.encoder.layers, activate=act_func)
        else:
            self.decoder = MLP(compressed, list(reversed(n_hidden_layers)) + [in_embeds], activate=act_func)
        
        
    
    def __call__(self, x):
        compressed = self.encoder(x)
        out = self.decoder(compressed)
        return out

    def parameters(self):
        return self.encoder.parameters() + self.decoder.parameters()

    def weights(self):
        return self.encoder.weights()

    def layers(self):
        return self.encoder.layers + self.decoder.layers

    def pretty(self):
        if self.act_func != None:
            hey = str(self.act_func)
            return hey.split()[1][6:]
        else:
            return "no function"

    def __repr__(self):
        return f"encoder has {self.summary()}, decoder has {self.summary()} activated with {self.pretty()}"




model = AutoEncoder(6, 1, 2, tied=True)

hey = [1.0, 2.0, 3.0, 4.0, 5.0]

print(model(hey))

[Value(data=-2.70118143540359, grad=0), Value(data=2.4023136894932335, grad=0), Value(data=-3.54287180637959, grad=0), Value(data=2.238346393859305, grad=0), Value(data=3.4033814120977426, grad=0), Value(data=-0.5534481657779167, grad=0)]


In [17]:
class Optimizer:
    """Base class for optimizers"""

    def __init__(self, parameters):
        self.parameters = parameters
    
    def zero_grad(self):
        for p in self.parameters:
            p.grad = 0

    def step(self):
        """Take a step of gradient descent"""

        raise NotImplementedError


class SGD(Optimizer):
    """Stochastic Gradient Descent optimizer"""

    def __init__(self, parameters, learning_rate=0.01):
        super().__init__(parameters)
        self.learning_rate = learning_rate

    def step(self):
        """Update model parameters in the opposite direction of their gradient"""

        for p in self.parameters:
            p.data -= self.learning_rate * p.grad

In [18]:
"""
Data utilities

Heavily inspired by https://github.com/joelgrus/joelnet/blob/master/joelnet/data.py
"""

# pylint: disable=too-few-public-methods

import random


# Batch = NamedTuple("Batch", [("inputs", List[Vector]), ("targets", Vector)])


class BatchIterator:
    """Iterates on data by batches"""

    def __init__(self,inputs,targets,batch_size=1, shuffle=True):
        self.inputs  = inputs
        self.targets = targets
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __call__(self):
        starts = list(range(0, len(self.inputs), self.batch_size))
        if self.shuffle:
            random.shuffle(starts)

        for start in starts:
            end = start + self.batch_size
            batch_inputs = self.inputs[start:end]
            batch_targets = self.targets[start:end]
            yield (batch_inputs, batch_targets)

In [31]:
def mean_squared_error(y_true, y_pred):
    """MSE loss"""

    total_squared_error = sum((y_true_i - y_pred_i)**2 for (y_true_i, y_pred_i) in zip(y_true, y_pred))
    n_total = max(len(y_true), 1)
    return total_squared_error / n_total

def binary_accuracy(y_true, y_pred):
    """Binary accuracy"""

    n_exact = sum(
        (
            y_true_i.data == round(y_pred_i.data)
            for (y_true_i, y_pred_i) in zip(y_true, y_pred)
        )
    )
    n_total: int = max(len(y_true), 1)
    return n_exact / n_total

def array_mean_squared_error(y_true, y_pred):
    """MSE loss"""

    total_squared_error = sum((y_true_i - y_pred_i)**2 for y_true_i, y_pred_i in zip(y_true, y_pred))
    n_total = max(len(y_true), 1)
    thingy = total_squared_error / 
    return sum(total_squared_error) / len(y_true[0])

In [57]:
# from pyfit.engine import Vector, Scalar
# from pyfit.nn import Module
# from pyfit.optim import Optimizer
# from pyfit.data import BatchIterator
# from pyfit.metrics import binary_accuracy

# Used to record training history for metrics
History = {}


class Trainer:
    """Encapsulates the model training loop"""

    def __init__(self, model, optimizer, loss):
        self.model = model
        self.optimizer = optimizer
        self.loss = loss

    def fit(self, data_iterator, num_epochs=500, verbose=False):
        """Fits the model to the data"""

        history = {"loss": [], "acc": []}
        epoch_loss = 0
        epoch_acc = 0
        epoch_y_true = []
        epoch_y_pred = []
        for epoch in range(num_epochs):
            # Reset the gradients of model parameters
            self.optimizer.zero_grad()
            # Reset epoch data
            epoch_loss = 0
            epoch_y_true = []
            epoch_y_pred = []

            for batch in data_iterator():
                # Forward pass
                outputs = list(map(self.model, batch[0]))
                

                # Loss computation
                # [item for sublist in outputs[0] for item in sublist]
                batch_y_pred = outputs
                batch_loss = self.loss(batch[1], batch_y_pred)
                print(batch_loss)
                epoch_loss += batch_loss.data

                # Store batch predictions and ground truth for computing epoch metrics
                epoch_y_pred.extend(batch_y_pred)
                epoch_y_true.extend(batch[1])

                # Backprop and gradient descent
                batch_loss.backward()
                self.optimizer.step()

            # Accuracy computation for epoch
            epoch_acc = binary_accuracy(epoch_y_true, epoch_y_pred)
            
            

            # Record training history
            history["loss"].append(epoch_loss)
            history["acc"].append(epoch_acc)

            if verbose:
                print(
                    f"Epoch [{epoch+1}/{num_epochs}], "
                    f"loss: {epoch_loss:.6f}, "
                    f"accuracy: {epoch_acc*100:.2f}%"
                )

        return history

In [58]:
model = AutoEncoder(64, 1, 2, tied=True)

hey = [1.0, 2.0, 3.0, 4.0, 5.0]

true = list(map(lambda x: Value(x), hey))


optimizer = SGD(model.parameters(), learning_rate=0.05)
data_iterator = BatchIterator(X_train, X_train, batch_size=32)
trainer = Trainer(model, optimizer, loss=array_mean_squared_error)

history = trainer.fit(data_iterator, num_epochs=50, verbose=True)

Value(data=91.87404562871508, grad=0)
Value(data=140653.8916777004, grad=0)
